# Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt


# Load the data

Download the data fron [here](https://storage.googleapis.com/biosense-ml-data/datasets/chest_xray.zip) and unzip it

In [ ]:
data_path = "https://storage.googleapis.com/biosense-ml-data/datasets/chest_xray.zip"

In [ ]:
# prompt: download the zip with images from data_path and unzip

!wget "https://storage.googleapis.com/biosense-ml-data/datasets/chest_xray.zip"
!unzip chest_xray.zip

In [ ]:
# Set the paths to your dataset
train_dir = 'chest_xray/train'
test_dir = 'chest_xray/test'
val_dir = 'chest_xray/val'


In [ ]:
# Load the image
image = plt.imread("/content/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg")

# Display the image
plt.imshow(image)
plt.show()


In [ ]:
# Load the image
image = plt.imread("/content/chest_xray/train/NORMAL/IM-0115-0001.jpeg")

# Display the image
plt.imshow(image)
plt.show()

In [ ]:
image.shape

In [ ]:
image

# Create TF datasets

In [ ]:
# Image dimensions and batch size
image_size = (256, 256)
batch_size = 32

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    class_names=['NORMAL', 'PNEUMONIA'],
    color_mode='grayscale',
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

In [ ]:
type(train_dataset)

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    class_names=['NORMAL', 'PNEUMONIA'],
    color_mode='grayscale',
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    class_names=['NORMAL', 'PNEUMONIA'],
    color_mode='grayscale',
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

# Review the data

In [ ]:
train_dataset

In [ ]:
train_dataset.take(1)

In [ ]:
train_dataset.take(1).as_numpy_iterator()

In [ ]:
first_batch = list(train_dataset.take(1).as_numpy_iterator())

In [ ]:
len(first_batch)

In [ ]:
# first_batch is a tuple of images and labels
len(first_batch[0])

In [ ]:
images, labels = first_batch[0]

In [ ]:
type(images)

In [ ]:
images.shape

In [ ]:
images[0]

In [ ]:
plt.imshow(images[31])

In [ ]:
plt.imshow(images[0])

# Convolutional Neural Network


In [ ]:
cnn_model = tf.keras.models.Sequential([

    # Input shape - (batch_size, height, width, channels)
    tf.keras.layers.Input(shape=(image_size[0], image_size[1], 1)),

    # tf.keras.layers.RandomFlip(mode="horizontal"),
    # tf.keras.layers.RandomBrightness(0.2, value_range=(0, 255)),

    # preprocessing
    tf.keras.layers.Rescaling(1./255),

    # Convolution block 1
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),

    # Convolution block 2
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),

    # Convolution block 3
    tf.keras.layers.Conv2D(filters=216, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=216, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),

    # Convolution block 4
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),

    # Feature extraction ends here

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'auc'])

In [ ]:
history = cnn_model.fit(
    train_dataset,
    epochs=2,
    validation_data=val_dataset
)

# Transfer learning - VGG16 Backbone

In [ ]:
image_size = (224, 224)

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    class_names=['NORMAL', 'PNEUMONIA'],
    color_mode='rgb',
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    class_names=['NORMAL', 'PNEUMONIA'],
    color_mode='rgb',
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    class_names=['NORMAL', 'PNEUMONIA'],
    color_mode='rgb',
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

In [ ]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3))

In [ ]:
base_model.layers

In [ ]:
base_model.summary()

In [ ]:
base_model.layers[1].weights[0].numpy().shape

In [ ]:
# display kernel at index 0
base_model.layers[1].weights[0].numpy()[:,:,:,0]

In [ ]:
temp = base_model.layers[1].weights[0].numpy()[:,:,:,30]

In [ ]:
temp[0]

In [ ]:
# displaying the kernel
plt.imshow(temp[1])

In [ ]:
i = list(train_dataset.take(1).as_numpy_iterator())[0][0][0:1]

In [ ]:
i.shape

In [ ]:
layer = base_model.layers[1]

In [ ]:
feature_map = layer(i)[0][:,:,0]

In [ ]:
plt.imshow(feature_map)

In [ ]:
# for layer in base_model.layers:
#     layer.trainable = False

base_model.trainable = False

flatten_output = layers.Flatten()(base_model.output)
hidden_dense_output = layers.Dense(512, activation='relu')(flatten_output)
dropout_output = layers.Dropout(0.5)(hidden_dense_output)
output_layer = layers.Dense(1, activation='sigmoid')(dropout_output)

model = Model(base_model.input, output_layer)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, dpi=80)

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=val_dataset
)

# Using GAP

In [ ]:
base_model.trainable = False

gap_output = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
dropout_output = layers.Dropout(0.2)(hidden_dense_output)
output_layer = layers.Dense(1, activation='sigmoid')(dropout_output)

gap_model = Model(base_model.input, output_layer)

In [ ]:
gap_model.summary()

In [ ]:
# Compile the model
gap_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = gap_model.fit(
    train_dataset,
    epochs=5,
    validation_data=val_dataset
)